### Projeto comparação entre algoritmos e modelos de Computação evolutiva

Achando o diretório em que esse arquivo se encontra 

In [1]:
import sys
import os

# Obtém o caminho do diretório do script atual
script_dir = os.getcwd()

# Adiciona o diretório pai ao caminho do sistema
sys.path.append(script_dir)

In [2]:
import pygame # Nos repositórios de referência essa lib é sempre importada tendo em vista que o gymnasium (gym) utiliza a mesma, deve evitar problemas

import time # será utilizada para coletar o período temporal necessário para que algumas funções sejam executadas
import numpy as np # numpy por motivos óbvios  

import torch # Todas as redes neurais serão criados utilizando PyTorch
import torch.nn as nn
import torch.optim as optim

from redes_neurais.rede_MLP import MLP
from func_aux.auxiliares import *

import matplotlib.pyplot as plt # Demonstração de Gráficos e imagens 

pygame 2.3.0 (SDL 2.24.2, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# usar GPU compatível com CUDA costuma ser mais rápido
if torch.cuda.is_available():
    is_gpu = torch.device("cuda")
else:
    is_gpu = torch.device("cpu")

torch.set_default_device(is_gpu)

In [4]:
import gymnasium as gym

In [5]:
def joga_jogo(env, rede_neural):

    if isinstance(env, str):
        env = gym.make(env)

    imagem, _ = env.reset()
    passos = 0
    game_recompenca = 0

    while True: # inicializa o loop do jogo
        passos += 1 # conta quantas vezes a IA foi requisitada ate o complecionismo do mesmo

        acao = escolhe_acao(rede_neural, imagem)

        nova_imagem, recompensa, finalizado, truncado, _ = env.step(acao)
        done = finalizado or truncado
        game_recompenca += recompensa

        imagem = nova_imagem

        if done:
            return rede_neural, passos, game_recompenca

In [8]:
# env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, NUM_STAPS
def treinamento_Foda(env, rede_neural_hidden, geracoes, n_por_geracao, n_selecionados = 2, pais = False, learning_rate = 0.001):
    
    if isinstance(env, str):
        env = gym.make(env)
    
    individuos = []
    
    input_dim = env.observation_space.shape[0]  # Número de entradas do ambiente MountainCar-v0
    output_dim = env.action_space.n  # Número de ações possíveis no ambiente MountainCar-v0

    for indi in range(n_por_geracao):
        individuos.append(MLP(input_dim, rede_neural_hidden, output_dim))

    lista_geracional = []
    recompencas = []

    for geracao in range(1,geracoes+1):
        contador = 0
        for especime_network in individuos:
            contador += 1

            modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
            lista_geracional.append([modelo_final, recompensa_total])
            recompencas.append(recompensa_total)
            # print(f'Geração: {geracao}/{geracoes}, Individuo: {contador}/{n_por_geracao}, Recompença: {recompensa_total}')
    
        pais_selecionados = seleciona_pais(lista_geracional, n_selecionados)

        print(f'Geração: {geracao}/{geracoes}, Recompensa: {sorted(recompencas, reverse=True)[:3]}')
        if pais:
            individuos = computacao_evolutiva_mutador_Cpais(n_por_geracao, pais_selecionados, learning_rate)
        else:
            individuos = computacao_evolutiva_mutador_Spais(n_por_geracao, pais_selecionados, learning_rate)
    
    for especime_network in individuos:
        contador += 1

        modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
        lista_geracional.append([modelo_final, recompensa_total])
        recompencas.append(recompensa_total)
        selecao_final = seleciona_pais(lista_geracional, 1)
        
    return selecao_final[0]

In [9]:
env = gym.make("CartPole-v1")
rede_melhor = treinamento_Foda(
    env = env,
    rede_neural_hidden = [36,36],
    geracoes = 10,
    n_por_geracao = 30,
    n_selecionados = 6,
    pais = True,
    learning_rate = 0.01
)

Geração: 1/10, Recompensa: [57.0, 55.0, 23.0]
Geração: 2/10, Recompensa: [338.0, 113.0, 69.0]
Geração: 3/10, Recompensa: [338.0, 296.0, 210.0]
Geração: 4/10, Recompensa: [500.0, 338.0, 296.0]
Geração: 5/10, Recompensa: [500.0, 500.0, 355.0]
Geração: 6/10, Recompensa: [500.0, 500.0, 500.0]
Geração: 7/10, Recompensa: [500.0, 500.0, 500.0]
Geração: 8/10, Recompensa: [500.0, 500.0, 500.0]
Geração: 9/10, Recompensa: [500.0, 500.0, 500.0]
Geração: 10/10, Recompensa: [500.0, 500.0, 500.0]


In [10]:
print(rede_melhor)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=4, out_features=36, bias=True)
    (1): ReLU()
    (2): Linear(in_features=36, out_features=36, bias=True)
    (3): ReLU()
    (4): Linear(in_features=36, out_features=2, bias=True)
  )
)


In [17]:
import os

def record_video_neuralnet(env, rede_neural, episodes=3, folder='videos/', prefix='CE-video', epsilon=0.0):
    """
    Grava um vídeo a partir de uma política epsilon-greedy definida pela 'qtable' e pelo valor de 'epsilon'.
    - env_name: A string do ambiente cadastrada no gymnasium ou uma instância da classe. Ao final, o ambiente é fechado (função `close()`).
    - qnet: A rede neural que representa a função Q.
    - episodes: Número de episódios completos que serão executados.
    - prefiz: Prefixo do nome dos arquivos de vídeo.
    - folder: Pasta onde os arquivos de vídeo serão salvos.
    - epsilon: Valor do parâmetro da política "epsilon-greedy" usada para escolher as ações.
    """
    if isinstance(env, str):
        env = gym.make(env, render_mode="rgb_array")

    rec_env = gym.wrappers.RecordVideo(env, folder, episode_trigger=lambda i : True, name_prefix=prefix)
    
    num_steps = 0
    for epi in range(episodes):
        imagem, _ = rec_env.reset()
        num_steps += 1
        epi_reward = 0.0
        done = False
        while not done:
            action = escolhe_acao(rede_neural, imagem)
            imagem, r, termi, trunc, _ = rec_env.step(action)
            done = termi or trunc
            num_steps += 1
            epi_reward += r
        print(f"Episode {epi}: {num_steps} steps / return {epi_reward:.2f}")
    rec_env.close()
    env.close()

    # Imprimir o caminho completo dos vídeos salvos
    for i in range(episodes):
        video_path = os.path.join(folder, f"{prefix}{i}.mp4")
        print(f"Caminho completo do vídeo {i}: {video_path}")



In [18]:
record_video_neuralnet(env, rede_melhor, 3)

Episode 0: 55 steps / return 54.00
Episode 1: 123 steps / return 67.00
Episode 2: 173 steps / return 49.00
Caminho completo do vídeo 0: videos/CE-video0.mp4
Caminho completo do vídeo 1: videos/CE-video1.mp4
Caminho completo do vídeo 2: videos/CE-video2.mp4
